In [39]:
!pip install paddlepaddle-gpu
!pip install paddleocr

In [40]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install paddleocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [79]:
import numpy as np
import pandas as pd
import re
import os
import spacy 
import cv2
from PIL import Image
from paddleocr import PaddleOCR,draw_ocr

In [141]:
DATASET_FOLDER = '/kaggle/input/testfile/'
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))


In [ ]:
test = test.copy()

test.loc[:, 'image_name'] = test['image_link'].str.split('/').str[-1]

test = test[:43729] #Akshat
# test = test[43729:87458] #Pradipto
# test = test[87458:131187] #Akshaay

test = test.drop(columns=['image_link'])

In [ ]:
len(test)

In [ ]:
ocr = PaddleOCR(use_angle_cls=True, lang='en')

In [ ]:
test_image_directory = '/kaggle/input/amazon-ml/test/'

def extract_text_from_image(image_path):
    result = ocr.ocr(image_path, cls=True)
    
    extracted_text = []
    if result[0] is not None:
        for line in result[0]:
            if len(line) > 0:
                extracted_text.append(line[1][0])
    
    return extracted_text

extracted_texts = []

for index, row in test.iterrows():
    image_file = row['image_name']  
    image_path = os.path.join(test_image_directory, f"{image_file}")  
    
    if os.path.exists(image_path):
        text = extract_text_from_image(image_path)
    else:
        text = ''  
    
    extracted_texts.append(text)


In [ ]:
ocr_list = []
for text in extracted_texts:
    ocr_list.append(" | ".join(text))

In [ ]:
test["ocr"]=ocr_list

In [ ]:
ocr_list

In [ ]:
test.to_csv('/kaggle/working/ocr_extracted_1.csv')

In [ ]:
DATASET_FOLDER = '/kaggle/input/ocr-files/'
ocr1 = pd.read_csv(os.path.join(DATASET_FOLDER, 'ocr_extracted_1.csv'))
ocr2 = pd.read_csv(os.path.join(DATASET_FOLDER, 'ocr_extracted_2.csv'))
ocr3 = pd.read_csv(os.path.join(DATASET_FOLDER, 'ocr_extracted_3.csv'))


In [ ]:
ocr_total = pd.concat([ocr1, ocr2, ocr3], axis=0)

ocr_total.to_csv('ocr_total.csv', index=False)

print("CSV files have been merged and saved as 'merged_file.csv'")


In [ ]:
ocr_total = ocr_total.copy()
ocr_total = ocr_total[:10]

In [ ]:
n = len(ocr_total)

ner_outputs = [None] * n

In [ ]:
import spacy
from spacy.matcher import Matcher
import re

# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')

# Entity-Unit map with abbreviations and full names
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Normalize abbreviations and common misspellings to their full forms
abbreviation_map = {
    'cm': 'centimetre', 'ft': 'foot', 'in': 'inch', 'm': 'metre', 'mm': 'millimetre', 'yd': 'yard',
    'g': 'gram', 'kg': 'kilogram', 'ug': 'microgram', 'mg': 'milligram', 'oz': 'ounce', 'lb': 'pound',
    'kv': 'kilovolt', 'mv': 'millivolt', 'v': 'volt', 'vlt': 'volt',  # Add common misspelling "vlt"
    'kw': 'kilowatt', 'w': 'watt',
    'cl': 'centilitre', 'cu ft': 'cubic foot', 'cu in': 'cubic inch', 'dl': 'decilitre', 'fl oz': 'fluid ounce',
    'gal': 'gallon', 'imperial gal': 'imperial gallon', 'l': 'litre', 'ul': 'microlitre', 'ml': 'millilitre', 'pt': 'pint', 'qt': 'quart'
}

# Initialize the matcher
matcher = Matcher(nlp.vocab)

# Create a reverse map for quick lookup from unit to entity
unit_to_entity = {unit: entity for entity, units in entity_unit_map.items() for unit in units}

# Define patterns to match entities with their respective units, allowing for fuzzy matching with 3 edits
for unit in unit_to_entity:
    pattern = [{"LIKE_NUM": True}, {"LOWER": {"FUZZY3": unit}}]  # Match numbers followed by fuzzy unit name (up to 3 edits)
    matcher.add(unit_to_entity[unit], [pattern])

# Text Preprocessing: Split text on "/" if it contains multiple measurements
def preprocess_text(text):
    text = re.sub(r'/', ' | ', text)
    return re.sub(r',', ' | ', text)  # Replace '/' with ' | ' to treat both sides as separate entities

def ner_parallel(index): 
    # Apply the SpaCy NLP pipeline to each text in the list
    # Preprocess the text by replacing "/" with a space or a separator
    
    if(pd.isna(ocr_total.iloc[index]["ocr"])==False):
        text = ocr_total.iloc[index]["ocr"]
        processed_text = preprocess_text(text)
        doc = nlp(processed_text)
        matches = matcher(doc)

        ner_output = []
        # Extract and print matched entities with labels, normalizing abbreviations and correcting misspellings
        for match_id, start, end in matches:
            span = doc[start:end]
            # Normalize abbreviations using abbreviation_map
            normalized_unit = abbreviation_map.get(span[-1].text.lower(), span[-1].text)
            temp_output = []
            # Loop through all possible entities for this unit and print them
            for entity, units in entity_unit_map.items():
                if normalized_unit in units:
                    temp_output.append([span[0].text + " " + normalized_unit,entity.replace('_', ' ').capitalize()])
            ner_output.append(temp_output)
        ner_outputs[index] = ner_output

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=128) as executor:
    results = list(tqdm(executor.map(ner_parallel, list(range(0, n))),total=n))

In [ ]:
ocr_total.head(20)

In [ ]:
n = len(ocr_total)

df = pd.DataFrame({
    'index': range(n),  # 0-indexed column
    'prediction': [""] * n  # Empty prediction column
})

In [ ]:
from tqdm import tqdm

# Example loop with tqdm progress bar
for i in tqdm(range(0, n)):
    ner_parallel(i)  # Call your function here


In [94]:
ocr_total.iloc[0][3]

'height'

In [128]:
ner_outputs[0][0][0]

['2.63 inch', 'Width']

In [129]:
def create_output(index):
    try:
        for ner_output in ner_outputs[index]:
            for lst in ner_output:
                if(ocr_total.iloc[index][3] == lst[1].lower()):
                    df.loc[index, 'prediction'] = lst[0]
                    break
    except:
        pass

In [133]:
# Example loop with tqdm progress bar
for i in (range(0, n)):
    create_output(i)  # Call your function here

In [138]:
df

,index,prediction
0,0,78 inch
1,1,200 centimetre
2,2,200 centimetre
3,3,200 centimetre
4,4,90 centimetre
...,...,...
131182,131182,
131183,131183,
131184,131184,
131185,131185,


In [148]:
df_concat = pd.concat([test["index"], df["prediction"]], axis=1)

In [155]:
def func(x):
    return x.replace(',', '.')

In [156]:
df_concat["prediction"]=df_concat["prediction"].apply(func)

In [157]:
df_concat.to_csv("output_draft2.csv",index=False)